<a href="https://colab.research.google.com/github/Toandinh1/DeepIM/blob/master/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%tensorflow_version 1.5

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.5`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [66]:
import numpy as np
import tensorflow as tf
from scipy.special import binom
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Conv1D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.python.ops.gen_math_ops import sign
#from keras import backend as K




N = 4 # number of sub-carriers
K = 2 # number of active sub-carriers
M = 4 # M-ary modulation order

SNRdb = 15 # Training SNR

traing_epochs = 100
l_rate = 0.1
total_batch = 20 # number of batches per epoch
batch_size = 100


n_hidden_1 = 16
n_hidden_2 = 32
n_hidden_3 = 32
n_hidden_4 = 16
n_input = 3*N
n_input_2 = 2*N + 2


m = int(np.log2(M))
c = int(np.log2(binom(N,K)))
q = K*m + c # number of bits per OFDM-IM symbol
Q= 2**q
n_output = c
n_output_2 = 4
c1 = 1
c2 = 0.25

pi = 3.14
Gl = 1
F = 2
d1 = 100
d2 = 400
PL1 = (4*pi*d1)/(np.sqrt(Gl)*((3*10^8)/F*1000))
PL2 = (4*pi*d2)/(np.sqrt(Gl)*((3*10^8)/F*1000))

display_step = 5
SNR = 10**(SNRdb/10)
sigma = np.sqrt(1/SNR)
qam_factor = (2/3)*(M-1)


a = 1/np.sqrt(2)

# M-ary modulations
if M==4:
    QAM = np.array([1+1j, 1-1j, -1+1j, -1-1j], dtype=complex) # gray mapping
elif M==8:
    QAM = np.array([1, a+a*1j, -a+a*1j, 1j, a-a*1j, -1j, -1, -a-a*1j], dtype=complex) # 8PSK, not 8QAM indeed
    qam_factor = 1
elif M==16:
    QAM = np.array([-3+3j, -3+1j, -3-3j, -3-1j, 
                    -1+3j, -1+1j, -1-3j, -1-1j, 
                    3+3j, 3+1j, 3-3j, 3-1j, 
                    1+3j, 1+1j, 1-3j, 1-1j], dtype=complex)
else:
    QAM = np.array([1, -1], dtype=complex) #BPSK
    qam_factor = 1




# index patterns for N=4 and K=1,2,3 only
if K==1:
    idx = np.array([[0],[1],[2],[3]])
elif K==2:
    idx = np.array([[0,1],[2,3],[0,2],[1,3]]) 
else:
    idx = np.array([[0,1,2],[1,2,3],[0,2,3],[0,1,3]]) 
def SC_IM_NO_train(bit1,bit2, SNRdb):
        #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = 1*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    Y1 = H1*np.sqrt(c1)*tx_sym1
    Y2 = H2*np.sqrt(c2)*tx_sym2
    
    Y = H1*np.sqrt(c1)*tx_sym1 + H2*np.sqrt(c2)*tx_sym2 + noise
     
    y_bar = Y/(np.sqrt(c1)*H1)
    y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)))
    y_m = np.absolute(Y1)
    y_con1 = np.concatenate((y_con,y_m))
    Ysic = Y - y_bar*H1*np.sqrt(c1)
    y2_ = Ysic/(np.sqrt(c2)*H2)
    y2_m = np.absolute(Y2)
    y2__ = np.concatenate((np.real(y2_),np.imag(y2_)))
    y_con2 =np.concatenate((y2__,y2_m))
    Y_con1= y_con1.reshape(n_input,)
    y_2 = y_con.reshape( n_input_2-2,)
  
    
    return y_con1,y_2

def SC_IM_NO_test(bit1, bit2, SNRdb):
        #user1
    bit_id1 = bit1[0:c:1]
    id_de1 = bit_id1.dot(2**np.arange(bit_id1.size)[::-1])
    bit_sy1 = bit1[c:q:1]   
    bit_K1 = bit_sy1.reshape(-1,m)
    sy_de1 = np.zeros((K,), dtype=int)
    sym1 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i1 = bit_K1[i,:]
        sy_de1[i] = bit_sy_i1.dot(2**np.arange(bit_sy_i1.size)[::-1])
        sym1[i] = QAM[sy_de1[i]]

    tx_sym1 = np.zeros((N,), dtype=complex)
    tx_sym1[idx[id_de1,:]] = sym1
    tx_sym1 = tx_sym1*np.sqrt(c1)
  #user2
    bit_id2 = bit2[0:c:1]
    id_de2 = bit_id2.dot(2**np.arange(bit_id2.size)[::-1])
    bit_sy2 = bit2[c:q:1]   
    bit_K2 = bit_sy2.reshape(-1,m)
    sy_de2 = np.zeros((K,), dtype=int)
    sym2 = np.zeros((K,), dtype=complex)
    for i in range(K):
        bit_sy_i2 = bit_K2[i,:]
        sy_de2[i] = bit_sy_i2.dot(2**np.arange(bit_sy_i2.size)[::-1])
        sym2[i] = QAM[sy_de2[i]]

    tx_sym2 = np.zeros((N,), dtype=complex)
    tx_sym2[idx[id_de2,:]] = sym2
    tx_sym2 = tx_sym2*np.sqrt(c2)

    #transmision
    SNR = 10**(SNRdb/10)
    sigma = np.sqrt(1/SNR)
    noise = sigma*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H1 = 4*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    H2 = 1*np.sqrt(1/2)*(np.random.randn(*tx_sym1.shape)+1j*np.random.randn(*tx_sym1.shape))
    Y1 = H1*np.sqrt(c1)*tx_sym1
    Y2 = H2*np.sqrt(c2)*tx_sym2
    
    Y = H1*np.sqrt(c1)*tx_sym1 + H2*np.sqrt(c2)*tx_sym2 + noise
     
    y_bar = Y/(np.sqrt(c1)*H1)
    y_con = np.concatenate((np.real(y_bar),np.imag(y_bar)))
    y_m = np.absolute(Y1)
    y_con1 = np.concatenate((y_con,y_m))
    Ysic = Y - y_bar*H1*np.sqrt(c1)
    y2_ = Ysic/(np.sqrt(c2)*H2)
    y2_m = np.absolute(Y2)
    y2__ = np.concatenate((np.real(y2_),np.imag(y2_)))
    y_con2 =np.concatenate((y2__,y2_m))
    Y_con1= y_con1.reshape(n_input,)
    y_2 = y_con.reshape(n_input_2 - 2,)
  
    
    return y_con1,y_2



In [60]:
model = Sequential()
model.add(Conv1D(32,3,activation='relu',input_shape=(12,1)))
model.add(Flatten())
model.add(Dense(64,activation='tanh'))
model.add(Dense(2,activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 10, 32)            128       
_________________________________________________________________
flatten_4 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                20544     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 130       
Total params: 20,802
Trainable params: 20,802
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.compile(loss="mse",optimizer='adam')
for epoch in range(traing_epochs):
  input_samples = []
  input_labels = []
  for index_k in range(0, 10000):
    bits = np.random.binomial(n=1,p=0.5,size=(q,))
    signal_output,signal_out_2 = SC_IM_NO_train(bits,bits,SNRdb)
    input_labels.append(bits[0:c:1])
    input_samples.append(signal_output)

  batch_x = np.asarray(input_samples)
  batch_x = batch_x.reshape(10000,12,1)
  batch_y = np.asarray(input_labels)
  model.fit(batch_x,batch_y,epochs=1)

Train on 10000 samples
10000/10000 [==============================] - 2s 181us/sample - loss: 4.2299e-05
Train on 10000 samples
10000/10000 [==============================] - 1s 54us/sample - loss: 3.2163e-07
Train on 10000 samples
10000/10000 [==============================] - 1s 56us/sample - loss: 3.4006e-08
Train on 10000 samples
10000/10000 [==============================] - 1s 65us/sample - loss: 2.5287e-06
Train on 10000 samples
10000/10000 [==============================] - 1s 53us/sample - loss: 3.9256e-05
Train on 10000 samples
10000/10000 [==============================] - 1s 56us/sample - loss: 3.7256e-05
Train on 10000 samples
10000/10000 [==============================] - 1s 62us/sample - loss: 9.8829e-05
Train on 10000 samples
10000/10000 [==============================] - 1s 56us/sample - loss: 2.4024e-06
Train on 10000 samples
10000/10000 [==============================] - 1s 66us/sample - loss: 3.8331e-05
Train on 10000 samples
10000/10000 [===========================

In [ ]:
#=========Training DNNs=============

X = tf.placeholder("float",[None, n_input_2])
Y = tf.placeholder("float",[None, n_output_2])
initializer = tf.contrib.layers.xavier_initializer()

def encoder(x):
    weights = {
        'encoder_h1': tf.Variable(initializer([n_input_2, n_hidden_1])),
        'encoder_h2': tf.Variable(initializer([n_hidden_1, n_output_2])),
    }
    
    biases = {
        'encoder_b1': tf.Variable(initializer([n_hidden_1])),
        'encoder_b2': tf.Variable(initializer([n_output_2])),
    }
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))

    return layer_2

y_pred = encoder(X)
y_true = Y

cost = tf.reduce_mean(tf.pow(y_true - y_pred,2))
learning_rate = tf.placeholder(tf.float32,shape=[])
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

def frange(x,y,jump):
    while x < y:
        yield x
        x +=jump

EbNodB_range = list(frange(0,50,5))
ber_info = [None]*len(EbNodB_range)
ber_index = [None]*len(EbNodB_range)

with tf.Session() as sess:
  #Training
    sess.run(init)
    for epoch in range(traing_epochs):
        avg_cost = 0
        for index_m in range(total_batch):
            input_samples = []
            input_labels = []
            input_samples_2 = []
            input_labels_2 = []
            for index_k in range(0, batch_size):
                bits = np.random.binomial(n=1,p=0.5,size=(q,))
                signal_output,signal_output_2 = SC_IM_NO_train(bits,bits,SNRdb)
                input_labels.append(bits[0:c:1])
                input_samples.append(signal_output)
                input_labels_2.append(bits[c::1])
                input_samples_2.append(signal_output_2)

            batch_x = np.asarray(input_samples)
            batch_y = np.asarray(input_labels_2)
            batch_x_2 = np.asarray(input_samples_2)
            batch_x = batch_x.reshape(batch_size,12,1)
            u_index = model.predict(batch_x)
            batch_x_2 = np.concatenate((u_index,batch_x_2),axis=1)
            

            _,cs = sess.run([optimizer,cost], feed_dict={X:batch_x_2,
                                                        Y:batch_y,
                                                        learning_rate:l_rate})
            avg_cost += cs / total_batch
        if epoch % display_step == 0:
            print("Epoch:",'%04d' % (epoch+1), "cost=", \
               "{:.9f}".format(avg_cost))
  #==========Testing=============
    for n in range(0,len(EbNodB_range)):
      input_samples_test = []
      input_labels_test = []
      input_samples_test_2 = []
      input_labels_test_2 = []
      for i in range(0, 100000):
        bits = np.random.binomial(n=1, p=0.5, size=(q, )) 
        signal_output,signal_output_2 = SC_IM_NO_test(bits,bits, EbNodB_range[n])
        input_labels_test.append(bits[0:c:1])
        input_samples_test.append(signal_output)
        input_labels_test_2.append(bits[c::1])
        input_samples_test_2.append(signal_output_2)
                
      batch_x_test = np.asarray(input_samples_test)
      batch_x_test_2 = np.asarray(input_samples_test_2)
      batch_y_test = np.asarray(input_labels_test_2)
      batch_y_test_2 = np.asarray(input_labels_test)
      batch_x_test = batch_x_test.reshape(100000,12,1)
      yhat = model.predict(batch_x_test)
      batch_x_test_2 = np.concatenate((yhat,batch_x_test_2),1)

      mean_error_rate = 1-tf.reduce_mean(tf.reduce_mean(tf.to_float(tf.equal(tf.sign(y_pred-0.5), tf.cast(tf.sign(batch_y_test-0.5),tf.float32))),1))
      ber_info[n]  = mean_error_rate.eval({X:batch_x_test_2})
      ber_index[n] = 1-tf.reduce_mean(tf.reduce_mean(tf.cast(tf.equal(tf.sign(yhat-0.5),tf.cast(tf.sign(batch_y_test_2-0.5),tf.float32)),tf.float32),1))
      

In [ ]:
ber_info

In [ ]:
ber_index